In [2]:
from utils import random_walk, generate_walks, read_train_val_graph, sparse_mx_to_torch_sparse_tensor
from utils import apply_word2vec_on_features, create_and_normalize_adjacency, save_subgraph_in_file, train_model

In [3]:
from random import random
from random import randint
from datetime import datetime
import torch

In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import pandas as pd
import numpy as np
from time import time
from gensim.models import Word2Vec
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
from unidecode import unidecode

def text_to_list(text):
    return unidecode(text).split(',')

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    is_common = 1 if len(lst3)>0 else 0
    return len(lst3), is_common
    

def add_authors_to_pairs (pairs, authors):
    np_pairs = np.array(pairs)

    pairs_df = pd.DataFrame(np.transpose(pairs)).rename(columns={0: "paper_1", 1: "paper_2"})
    pairs_df = pairs_df.merge(authors, left_on='paper_1', right_on='paper_id', how='left').rename(columns={'authors': "authors_1"})
    pairs_df = pairs_df.merge(authors, left_on='paper_2', right_on='paper_id', how='left').rename(columns={'authors': "authors_2"})
    pairs_df.drop(['paper_id_x', 'paper_id_y'], axis=1, inplace=True)

    pairs_df['nb_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[0], axis=1)
    pairs_df['is_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[1], axis=1)

    pairs_tensor = torch.LongTensor(np.transpose(pairs_df[["paper_1", "paper_2", 'is_common_author', 'nb_common_author']].values.tolist()))
    
    return pairs_tensor



In [80]:
from random import choice


def random_walk(G, node, walk_length):
    walk = [node]
  
    for i in range(walk_length-1):
        neibor_nodes = list(G.neighbors(walk[-1]))
        if len(neibor_nodes) > 0:
            next_node = choice(neibor_nodes)
            walk.append(next_node)
    walk = [node for node in walk] # in case the nodes are in string format, we don't need to cast into string, but if the nodes are in numeric or integer, we need this line to cast into string
    return walk


def generate_walks(G, num_walks, walk_length):
  # Runs "num_walks" random walks from each node, and returns a list of all random walk
    t = time()
    print('Start generating walks....')
    walks = list()  
    for i in range(num_walks):
        for node in G.nodes():
            walk = random_walk(G, node, walk_length)
            walks.append(walk)
        #print('walks : ', walks)
    print('Random walks generated in in {}s!'.format(round(time()-t)))
    return walks

def apply_word2vec_on_features(features, nodes, vector_size=128, window=5, min_count=0, sg=1, workers=8):
    t = time()
    print('Start applying Word2Vec...')
    wv_model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=workers)
    wv_model.build_vocab(features)
    wv_model.train(features, total_examples=wv_model.corpus_count, epochs=5) 
    print('Word2vec model trained on features in {} min!'.format(round((time()-t)/60)))
    features_np = []
    for node in nodes:
        features_np.append(wv_model.wv[node])

    features_np = np.array(features_np)
    print(features_np.shape, 'features numpy array created in {} min!'.format(round((time()-t)/60)))
    return features_np

In [8]:
vec = torch.LongTensor([1, 0, 4])

mat = torch.LongTensor([[1, 3, 4], 
                        [4, 6, 7], 
                        [2, 3, 4]])

vec[:, None]*mat

tensor([[ 1,  3,  4],
        [ 0,  0,  0],
        [ 8, 12, 16]])

In [81]:
path = '../input_data/small_edgelist.txt'

G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx = read_train_val_graph(val_ratio=0.1, path=path)
walks = generate_walks(G=G_train, num_walks=10, walk_length=15)
walks_wv = apply_word2vec_on_features(features=walks, nodes=nodes, vector_size=64)
adj, indices = create_and_normalize_adjacency(G_train)

Number of nodes: 980 number of edges: 2405 in All the set
Number of nodes: 980 number of edges: 2156 in the Training set
len(nodes) 980
Creating random val_edges...
Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects
Loaded from small_edgelist.txt and with a training validation split ratio = 0.1
Start generating walks....
Random walks generated in in 0s!
Start applying Word2Vec...
Word2vec model trained on features in 0 min!
(980, 64) features numpy array created in 0 min!
Created a normalized adjancency matrix of shape (980, 980)
Created indices (2, 5292) with the positions of non zeros in adj matrix


/Users/ghassenabdedayem/Documents/Data/Polytechnique/5- Data Challenge/data_challenge_2022/draft_nb/utils.py:146: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph


In [6]:
counter = 0
non_counter = 0
for val_edge in val_edges:
    if val_edge in G.edges():
        counter += 1
    else:
        non_counter += 1
print('val_edges in G', counter)
print('val_edges not in G', non_counter)

counter = 0
for val_edge in val_edges:
    if val_edge in G_train.edges():
        counter += 1
print('val_edges in G_train', counter)

val_edges in G 235
val_edges not in G 233
val_edges in G_train 1


In [72]:
#authors_np.tolist()[0]

In [83]:
walks

[[0, 1, 5, 1, 3, 1, 15, 1, 22, 1, 3, 1, 20, 23, 1],
 [1, 23, 1, 14, 1, 11, 1, 13, 1, 6, 16, 10, 16, 1, 17],
 [2, 41, 33, 43, 425, 464, 405, 420, 37, 2, 41, 407, 597, 618, 671],
 [3, 1, 19, 1, 3, 1, 3, 1, 14, 1, 9, 21, 9, 21, 9],
 [5, 1, 10, 16, 10, 12, 1, 10, 13, 1, 13, 10, 14, 1, 6],
 [6, 1, 20, 23, 1, 17, 1, 6, 16, 6, 5, 1, 3, 1, 7],
 [7, 1, 23, 1, 6, 16, 10, 13, 1, 23, 20, 23, 1, 13, 1],
 [9, 1, 6, 1, 5, 1, 11, 1, 3, 1, 0, 2, 53, 660, 695],
 [10, 13, 10, 14, 1, 17, 1, 23, 20, 1, 5, 1, 9, 21, 9],
 [11, 1, 14, 1, 14, 1, 22, 1, 13, 1, 16, 6, 16, 10, 1],
 [12, 1, 11, 1, 14, 1, 16, 10, 14, 1, 14, 10, 14, 10, 14],
 [13, 1, 3, 1, 20, 23, 20, 23, 1, 20, 1, 11, 1, 17, 1],
 [14, 1, 19, 1, 7, 1, 9, 1, 22, 1, 22, 1, 9, 1, 5],
 [15, 1, 19, 1, 11, 1, 20, 1, 23, 1, 13, 1, 22, 1, 22],
 [16, 6, 5, 1, 13, 1, 6, 5, 1, 22, 1, 17, 1, 11, 1],
 [17, 1, 23, 1, 17, 1, 20, 1, 12, 1, 13, 1, 20, 1, 14],
 [19, 1, 5, 6, 16, 1, 14, 1, 9, 1, 15, 1, 16, 10, 16],
 [20, 1, 23, 1, 13, 1, 15, 1, 22, 1, 23, 20, 1, 19, 1

In [86]:
from sklearn.preprocessing import MultiLabelBinarizer
from gensim.models import Word2Vec


authors = pd.read_csv('../input_data/authors.txt', sep = '|', header=None)
authors = authors.rename(columns={0: "paper_id", 2: "authors"})
authors['authors'] = authors['authors'].apply(text_to_list)
authors = authors[["paper_id", "authors"]]
authors = authors[authors['paper_id'] <= max(G.nodes())]
mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(authors['authors']),columns=mlb.classes_, index=authors.index)
authors_np = df.values

wv_model = Word2Vec(vector_size=64, window=5, min_count=0, sg=1, workers=8)
wv_model.build_vocab(authors_np)
wv_model.train(authors_np.tolist(), total_examples=wv_model.corpus_count, epochs=5) 

#authors_wv = apply_word2vec_on_features(authors_np, nodes, vector_size=64)

(435589, 12885000)

In [118]:
wv_model.wv.
#index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.

TypeError: get_vecattr() missing 2 required positional arguments: 'key' and 'attr'

In [82]:
from sklearn.preprocessing import MultiLabelBinarizer

authors = pd.read_csv('../input_data/authors.txt', sep = '|', header=None)
authors = authors.rename(columns={0: "paper_id", 2: "authors"})
authors['authors'] = authors['authors'].apply(text_to_list)
authors = authors[["paper_id", "authors"]]
authors = authors[authors['paper_id'] <= max(G.nodes())]

mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(authors['authors']),columns=mlb.classes_, index=authors.index)
authors_np = df.values

authors_wv = apply_word2vec_on_features(authors_np.tolist(), df.index.tolist(), vector_size=64)

Start applying Word2Vec...
Word2vec model trained on features in 0 min!


KeyError: "Key '2' not present"

In [ ]:
features_np = np.concatenate(walks_wv, authors_wv, axis=1)

In [ ]:
# Create class labels
y = np.zeros(2*indices.shape[1])
y[:indices.shape[1]] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.

# Transforms the numpy matrices/vectors to torch tensors.
features = torch.FloatTensor(features_np).to(device)
y = torch.LongTensor(y).to(device)
if type(adj) != torch.Tensor:
    adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
indices = torch.LongTensor(indices).to(device)

In [ ]:
class GNN(nn.Module):
    def __init__(self, n_feat, n_hidden, n_class, dropout):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(n_feat, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden+2, n_hidden)
        self.double_fc3 = nn.Linear((2*n_hidden), n_hidden)
        self.fc4 = nn.Linear(n_hidden, n_class)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
        

    def forward(self, x_in, adj, pairs):
        
        h1 = self.fc1(x_in)
        z1 = self.relu(torch.mm(adj, h1))
        z1 = self.dropout(z1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.mm(adj, h2))
        z2 = self.dropout(z2)

        x = z2[pairs[0]] - z2[pairs[1]] # embedded features (z2) of node 0 - embedded features of node 1
        
        x_auth = 4*pairs[2].reshape([len(pairs[2]), 1])
        x_nb_auth = 2*pairs[3].reshape([len(pairs[3]), 1]) 
        
        x = torch.cat((x, x_auth, x_nb_auth), dim=1) #pairs[3] : number of same authors or 1 if same author
        
        #x1 = z2[pairs[0]]
        #x2 = z2[pairs[1]]
        # could we add a new dimension to pairs to specify if same author(s)? and then what could we do?
              
        #x = torch.cat((x1, x2), dim=1)        
        #x = self.relu(self.double_fc3(x))        
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return F.log_softmax(x, dim=1)

In [ ]:
epochs = 101
n_hidden = 128
dropout_rate = 0.2
n_class = 2
n_features = features.shape[1]

# Creates the model and specifies the optimizer
model = GNN(n_features, n_hidden, n_class, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train_model(model, optimizer, features, authors, adj, indices, y, val_indices, y_val, epochs):
    # Train model
    model.train()
    start_time = time()
    val_indices = add_authors_to_pairs(val_indices, authors) #we add the authors to val_pairs
    indices = add_authors_to_pairs(indices, authors) #we add the authors to indices
    today = datetime.today().strftime('%Y-%m-%d')
    for epoch in range(epochs):
        t = time()
        optimizer.zero_grad()
        rand_indices = torch.randint(0, features.size(0), (indices.size(0),indices.size(1)), device=adj.device)# We take random indices each time we run an epoch
        rand_indices = add_authors_to_pairs(rand_indices, authors)
        pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices.   
        #pairs = add_authors_to_pairs(pairs) #we add the authors to the pairs
        output = model(features, adj, pairs) # we run the model that gives the output.
        loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
        acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
        loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, adj, val_indices)
        y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())

        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {:.4f} s'.format(round(time() - t)),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
        if epochs % 50 == 0:
            model_path = "../outputs/models/{}-model-{}epochs.pt".format(today, epoch)
            torch.save(model.state_dict(), model_path)


    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model

In [ ]:
trained_model = train_model(model, optimizer, features, adj, indices, y, val_indices, y_val, epochs)
break

In [ ]:
def train_model(model, optimizer, features, adj, indices, y, val_indices, y_val, epochs):
    # Train model
    model.train()
    start_time = time()
    val_indices = add_authors_to_pairs(val_indices) #we add the authors to val_pairs
    indices = add_authors_to_pairs(indices) #we add the authors to indices
    rand_indices = torch.randint(0, features.size(0), (indices.size(0),indices.size(1)), device=adj.device)# We take random indices each time we run an epoch
    rand_indices = add_authors_to_pairs(rand_indices)
    today = datetime.today().strftime('%Y-%m-%d')
    for epoch in range(epochs):
        t = time()
        optimizer.zero_grad()
        pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices.   
        output = model(features, adj, pairs) # we run the model that gives the output.
        loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
        acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
        loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, adj, val_indices)
        y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())

        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {:.4f} s'.format(round(time() - t)),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
        if epochs % 50 == 0:
            model_path = "../outputs/models/{}-model-{}epochs.pt".format(today, epoch)
            torch.save(model.state_dict(), model_path)


    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model

epochs = 100
trained_model = train_model(model, optimizer, features, adj, indices, y, val_indices, y_val, epochs)


In [ ]:
break
#y_val = torch.FloatTensor(y_val).to(device)
trained_model = train_model(model, optimizer, features, adj, indices, y, val_indices, y_val, epochs)
model_nb = randint(0, 1000)
today = datetime.today().strftime('%Y-%m-%d')
model_path = "../submissions_files/{}-model-{}epochs-{}.pt".format(today, epochs, model_nb)
torch.save(trained_model.state_dict(), model_path)
print('Model saved in', model_path)

In [ ]:
model.eval()
output = model(features, adj, val_indices)
y_val = torch.LongTensor(y_val).to(device)
loss_val = F.nll_loss(output, y_val)
acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
print(loss_val.item())

In [ ]:
# Evaluating the model
trained_model.eval()
eval_pairs = np.array(np.transpose(val_edges))
#print(eval_pairs.shape)
eval_pairs = torch.LongTensor(eval_pairs).to(device)
#print(eval_pairs.shape)
eval_output = trained_model(features, adj, eval_pairs)
#print(eval_output.shape)
y_pred = torch.exp(eval_output)
y_pred = y_pred.detach().cpu().numpy()

#y_val_pred_true = list()

y_val_pred_true = y_pred[:, 1]

    
print('Log loss:', log_loss(y_val, y_val_pred_true))

#y_val = torch.tensor(y_val).to(device)
#y_val_pred_true = torch.tensor(y_val_pred_true).to(device)
#print(y_val, y_val_pred_true)

In [ ]:
from datetime import datetime
import pandas as pd



# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('../test_data/test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
pairs = torch.LongTensor(node_pairs).to(device)
output = model(features, adj, pairs)
y_pred = torch.exp(output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])
    

    
today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"../submissions_files/{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
break

In [ ]:
y_val = y_val.detach().cpu().numpy()
y_val

In [ ]:
df = pd.DataFrame(y_pred)
df['y_val'] = list(y_val)
df.to_csv('../submissions_files/comparison_file.csv', sep=';')

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_val, y_pred[:, 1]), mean_absolute_error(y_val, y_pred[:, 0])

In [ ]:
log_loss(y_val, y_pred[:, 1]), log_loss(y_val, y_pred[:, 0])

In [ ]:
test_features = np.array([[5, 3, 0], [2, 1, 0], [4, 0, 1], [0, 3, 2]])
test_features = torch.LongTensor(test_features).to(device)
test_pairs = [[0, 0, 1, 2, 3, 2, 0, 1, 2, 3], [1, 2, 0, 0, 2, 3, 0, 1, 2, 3]]
test_pairs = torch.LongTensor(test_pairs).to(device)
print(test_features.shape, test_pairs.shape)
test_features[test_pairs[0]]

In [ ]:
y_pred.shape, np.shape(y_val), np.shape(y_pred_true)

In [ ]:
#print(output.shape)
print(features.shape, adj.shape, eval_pairs.shape)

In [ ]:
z2[pairs[1,:]]

In [ ]:
break

In [ ]:
break

In [ ]:
# Create class labels
y = np.zeros(4*G_train.number_of_edges())
y[:2*G_train.number_of_edges()] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.
train_edges[0]

In [ ]:
t = time()
X_train = np.zeros((4*m, 2*features_np.shape[1]))

for i, edge in enumerate(train_edges):
    X_train[i] = np.concatenate((features_np[train_edges[i][0]], features_np[train_edges[i][1]]), axis=0)
    X_train[m+i] = np.concatenate((features_np[train_edges[i][0]], features_np[train_edges[i][1]]), axis=0)
    X_train[2*m+i] = np.concatenate((features_np[randint(0,n-1)], features_np[randint(0,n-1)]), axis=0)
    X_train[3*m+i] = np.concatenate((features_np[randint(0,n-1)], features_np[randint(0,n-1)]), axis=0)
    
X_val = np.zeros((len(val_edges), 2*features_np.shape[1]))
for i, edge in enumerate(val_edges):
    X_val[i] = np.concatenate((features_np[val_edges[i][0]], features_np[val_edges[i][1]]), axis=0)
    
print('X_train and X_val created in {} s!'.format(round(time()-t)))

In [ ]:
t = time()
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y)
y_pred = clf.predict_proba(X_val)
y_pred = y_pred[:,1]
print('Logistic regression performed in {} s!'.format(round(time()-t)))

In [ ]:
print('Log loss:', log_loss(y_val, y_pred))

In [ ]:
from datetime import datetime

# Read test data. Each sample is a pair of nodes
test_edges = list()
with open('../test_data/test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        test_edges.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

X_test = np.zeros((len(test_edges), 2*features_np.shape[1]))
for i, edge in enumerate(test_edges):
    X_test[i] = np.concatenate((features_np[test_edges[i][0]], features_np[test_edges[i][1]]), axis=0)

t = time()
# Use logistic regression to predict if two nodes are linked by an edge
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]
print('Logistic regression performed in {} s!'.format(round(time()-t)))


today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 100000)

pd.DataFrame(y_pred, columns={'predicted'}).to_csv(
"../submissions_files/{}-submission-{}.csv".format(today, random_nb), header=True, index=True, index_label='id'
)
    
    
# # Testing
# node_pairs = np.array(np.transpose(node_pairs))
# pairs = torch.LongTensor(node_pairs).to(device)
# output = model(features, adj, pairs)
# y_pred = torch.exp(output)
# y_pred = y_pred.detach().cpu().numpy()

# y_pred_true = list()
# for element in y_pred:
#     y_pred_true.append(element[1])
    

    
# today = datetime.today().strftime('%Y-%m-%d')
# random_nb = randint(0, 100000)

# pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
# "../submissions_files/{}-submission-{}.csv".format(today, random_nb), header=True, index=True, index_label='id'
# )